In [1]:
import pandas as pd
import os

# Define the base path
base_path = r"C:\Users\SURA\OneDrive\Desktop\UKHM  Land Registry PPD 2021-2023"

# List of CSV files
csv_files = [
    os.path.join(base_path, "pp-2021.csv"),
    os.path.join(base_path, "pp-2022.csv"),
    os.path.join(base_path, "pp-2025.csv"),
    os.path.join(base_path, "pp-2024.csv"),
    os.path.join(base_path, "pp-2023.csv")
]

# Define column names for HM Land Registry Price Paid Data
columns = [
    "transaction_id",         # Unique transaction GUID
    "price",                  # Sale price (£)
    "date_of_transfer",       # Date of sale
    "postcode",               # Postcode
    "property_type",          # D=Detached, S=Semi, T=Terraced, F=Flat, O=Other
    "old_new",                # Y=New build, N=Old
    "duration",               # F=Freehold, L=Leasehold
    "paon",                   # Primary addressable object
    "saon",                   # Secondary addressable object
    "street",                 # Street name
    "locality",               # Optional sub-locality
    "town_city",              # Town or city
    "district",               # Local authority district
    "county",                 # County
    "ppd_category_type",      # A=Standard, B=Additional
    "record_status"           # A=Added, C=Changed
]

# Load and combine all CSV files into one DataFrame with defined columns
dataframes = [pd.read_csv(file, header=None, names=columns, encoding='utf-8', low_memory=False) for file in csv_files]
combined_df = pd.concat(dataframes, ignore_index=True)

# Print details to verify
print(f"Combined DataFrame shape: {combined_df.shape}")
print(f"Combined DataFrame columns: {combined_df.columns.tolist()}")
print(f"Sample of combined data:\n{combined_df.head()}\n")

# Save the combined DataFrame to a new CSV file
output_file = os.path.join(base_path, "combined_ukhm.csv")
combined_df.to_csv(output_file, index=False)
print(f"Saved combined data to {output_file}")

print("Extraction and combination complete!")

Combined DataFrame shape: (4535701, 16)
Combined DataFrame columns: ['transaction_id', 'price', 'date_of_transfer', 'postcode', 'property_type', 'old_new', 'duration', 'paon', 'saon', 'street', 'locality', 'town_city', 'district', 'county', 'ppd_category_type', 'record_status']
Sample of combined data:
                           transaction_id   price  date_of_transfer  postcode  \
0  {CB0035E6-3546-58AE-E053-6B04A8C091AF}  630000  2021-04-29 00:00  BN13 3AH   
1  {CB0035E6-3547-58AE-E053-6B04A8C091AF}  477000  2021-05-10 00:00  BN16 2PQ   
2  {CB0035E6-3548-58AE-E053-6B04A8C091AF}  370000  2021-05-06 00:00   BN2 4HZ   
3  {CB0035E6-3549-58AE-E053-6B04A8C091AF}  462500  2021-04-28 00:00   BN2 0GP   
4  {CB0035E6-354A-58AE-E053-6B04A8C091AF}  433000  2021-08-06 00:00   BN8 4LS   

  property_type old_new duration paon saon           street    locality  \
0             D       N        F    3  NaN  HIGHLANDS CLOSE         NaN   
1             D       N        F   10  NaN   CHAUCER AVENUE

In [2]:
import pandas as pd
import os

# Input and output file paths
input_file = r"C:\Users\SURA\OneDrive\Desktop\UKHM  Land Registry PPD 2021-2023\combined_ukhm.csv"
output_file = r"C:\Users\SURA\OneDrive\Desktop\UKHM  Land Registry PPD 2021-2023\combined_ukhm_clean.csv"

# Define expected column names
expected_columns = [
    "transaction_id",         # Unique transaction GUID
    "price",                  # Sale price (£)
    "date_of_transfer",       # Date of sale
    "postcode",               # Postcode
    "property_type",          # D=Detached, S=Semi, T=Terraced, F=Flat, O=Other (to be transformed)
    "old_new",                # Y=New build, N=Old
    "duration",               # F=Freehold, L=Leasehold
    "paon",                   # Primary addressable object
    "saon",                   # Secondary addressable object
    "street",                 # Street name
    "locality",               # Optional sub-locality
    "town_city",              # Town or city
    "district",               # Local authority district
    "county",                 # County
    "ppd_category_type",      # A=Standard, B=Additional
    "record_status"           # A=Added, C=Changed
]

# Mapping dictionaries for all encoded columns
property_type_map = {
    "D": "Detached",
    "S": "Semi-Detached",
    "T": "Terraced",
    "F": "Flat/Maisonette",
    "O": "Other"
}

old_new_map = {
    "Y": "New Build",
    "N": "Established"
}

duration_map = {
    "F": "Freehold",
    "L": "Leasehold"
}

ppd_category_map = {
    "A": "Standard Price Paid Entry",
    "B": "Additional Price Paid Entry"
}

record_status_map = {
    "A": "Added",
    "C": "Changed",
    "D": "Deleted"
}

# Load the combined CSV file
df = pd.read_csv(input_file, encoding='utf-8', low_memory=False)

print("--- Initial Data Overview ---")
print(f"Original shape: {df.shape}")

# 1. Identify and report missing values BEFORE any major drops/transformations
print("\n--- Missing Values Before Column Drops ---")
missing_values_count_initial = df.isnull().sum()
missing_values_percent_initial = 100 * df.isnull().sum() / len(df)
initial_missing_df = pd.DataFrame({
    'Missing Count': missing_values_count_initial,
    'Missing Percent': missing_values_percent_initial
})
initial_missing_df = initial_missing_df[initial_missing_df['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False)
if not initial_missing_df.empty:
    print(initial_missing_df)
else:
    print("No missing values found initially.")

# 2. Identify columns to drop based on a threshold (e.g., 60%)
missing_threshold = 60
cols_to_drop = initial_missing_df[initial_missing_df['Missing Percent'] >= missing_threshold].index.tolist()

if cols_to_drop:
    print(f"\n--- Dropping Columns with >= {missing_threshold}% Missing Values ---")
    print(f"Columns to drop: {cols_to_drop}")
    df.drop(columns=cols_to_drop, inplace=True)
    print(f"Shape after dropping high-missing columns: {df.shape}")
else:
    print(f"\nNo columns found with >= {missing_threshold}% missing values.")

# 3. Identify and report duplicate rows
print("\n--- Duplicate Rows Before Cleaning ---")
initial_duplicates = df.duplicated().sum()
if initial_duplicates > 0:
    print(f"Found {initial_duplicates} duplicate rows.")
else:
    print("No duplicate rows found initially.")

# --- Perform Transformations ---
print("\n--- Performing Data Transformations ---")
df["price"] = pd.to_numeric(df["price"], errors="coerce")          # Convert price to numeric
df["date_of_transfer"] = pd.to_datetime(df["date_of_transfer"], errors="coerce")  # Convert date

# ✅ Apply mapping for categorical codes to readable labels
if "property_type" in df.columns:
    df["property_type"] = df["property_type"].map(property_type_map).fillna(df["property_type"])

if "old_new" in df.columns:
    df["old_new"] = df["old_new"].map(old_new_map).fillna(df["old_new"])

if "duration" in df.columns:
    df["duration"] = df["duration"].map(duration_map).fillna(df["duration"])

if "ppd_category_type" in df.columns:
    df["ppd_category_type"] = df["ppd_category_type"].map(ppd_category_map).fillna(df["ppd_category_type"])

if "record_status" in df.columns:
    df["record_status"] = df["record_status"].map(record_status_map).fillna(df["record_status"])

# 4. Drop rows with missing 'price' or 'date_of_transfer' AFTER conversion
print(f"\nShape before dropping rows with missing 'price' or 'date_of_transfer': {df.shape}")
rows_before_dropping_critical_na = df.shape[0]
df = df.dropna(subset=["price", "date_of_transfer"])
rows_after_dropping_critical_na = df.shape[0]
print(f"Dropped {rows_before_dropping_critical_na - rows_after_dropping_critical_na} rows due to missing 'price' or 'date_of_transfer'.")
print(f"Shape after dropping rows with missing 'price' or 'date_of_transfer': {df.shape}")

# 5. Handle duplicates after cleaning
print("\n--- Handling Duplicate Rows (Post-Transformation) ---")
original_rows_post_transform = df.shape[0]
df.drop_duplicates(inplace=True)
rows_after_dropping_duplicates = df.shape[0]
duplicates_removed = original_rows_post_transform - rows_after_dropping_duplicates
if duplicates_removed > 0:
    print(f"Removed {duplicates_removed} duplicate rows.")
else:
    print("No duplicate rows to remove after transformations and critical NA drop.")

# --- Final Reporting ---
print("\n--- Final Data Overview ---")
print("✅ CSV Loaded and Transformed Successfully!")
print("Shape:", df.shape)
print("Column names:", df.columns.tolist())

print("\n🏠 Price Summary:")
print(df["price"].describe())

print("\nProperty Type Distribution:")
print(df["property_type"].value_counts())

# Report missing values after ALL cleaning
print("\n--- Missing Values After ALL Cleaning ---")
missing_values_count_final = df.isnull().sum()
missing_values_percent_final = 100 * df.isnull().sum() / len(df)
final_missing_df = pd.DataFrame({
    'Missing Count': missing_values_count_final,
    'Missing Percent': missing_values_percent_final
})
final_missing_df = final_missing_df[final_missing_df['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False)
if not final_missing_df.empty:
    print(final_missing_df)
else:
    print("No missing values remaining after all cleaning steps.")

# Save the clean DataFrame
df.to_csv(output_file, index=False, encoding='utf-8')
print(f"\nTransformation complete! Cleaned data saved to {output_file}")


--- Initial Data Overview ---
Original shape: (4535701, 16)

--- Missing Values Before Column Drops ---
          Missing Count  Missing Percent
saon            3939136        86.847347
locality        2807270        61.892748
street            77967         1.718963
postcode          12179         0.268514

--- Dropping Columns with >= 60% Missing Values ---
Columns to drop: ['saon', 'locality']
Shape after dropping high-missing columns: (4535701, 14)

--- Duplicate Rows Before Cleaning ---
No duplicate rows found initially.

--- Performing Data Transformations ---

Shape before dropping rows with missing 'price' or 'date_of_transfer': (4535701, 14)
Dropped 0 rows due to missing 'price' or 'date_of_transfer'.
Shape after dropping rows with missing 'price' or 'date_of_transfer': (4535701, 14)

--- Handling Duplicate Rows (Post-Transformation) ---
No duplicate rows to remove after transformations and critical NA drop.

--- Final Data Overview ---
✅ CSV Loaded and Transformed Successfull

In [3]:
# This file will be the input for our load_to_db.py script.

output_file = 'combined_ukhm_clean.parquet' # <-- Correct the filename and extension
print(f"Saving clean data to '{output_file}'...")

df.to_parquet(output_file, index=False) # This will now save a proper .parquet file

print("Clean data saved successfully. You can now run the load_to_db.py script.")

Saving clean data to 'combined_ukhm_clean.parquet'...
Clean data saved successfully. You can now run the load_to_db.py script.
